# About
TFLite does not support dynamic batch size, so we need to use static batch size.

This notebook convert the MobileNetEdgeTPU from TF to TFLite with a static batch size for batch inference.

Download and extract this archive (wget does not work with this link):

https://storage.cloud.google.com/mobilenet_edgetpu/checkpoints/mobilenet_edgetpu_224_1.0.tgz

Then upload the file **frozen_graph_tf1x.pb** into this notebook.

# Convert TF to TFLite

In [1]:
import tensorflow as tf
print(tf.__version__)

2.6.0


In [4]:
batch_size = 3
frozen_graph_file = 'frozen_graph_tf1x.pb'
tflite_file = f'mobilenet_edgetpu_{batch_size}x224x224x3_1.0_float.tflite'

converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file=frozen_graph_file, 
    input_arrays=['input'],
    output_arrays=['MobilenetEdgeTPU/Predictions/Softmax'],
    input_shapes={'input': [batch_size, 224, 224, 3]}
)
tflite_model = converter.convert()

with open(tflite_file, 'wb') as f:
  f.write(tflite_model)


# Validate the converted TFLite model

In [10]:
!wget https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/examples/label_image/testdata/grace_hopper.bmp
!curl https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_1.0_224_frozen.tgz  | tar xzv -C ./  mobilenet_v1_1.0_224/labels.txt

--2021-08-20 10:01:59--  https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/examples/label_image/testdata/grace_hopper.bmp
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 940650 (919K) [image/bmp]
Saving to: ‘grace_hopper.bmp.2’

grace_hopper.bmp.2  100%[===================>] 918.60K  --.-KB/s    in 0.03s   

2021-08-20 10:01:59 (26.7 MB/s) - ‘grace_hopper.bmp.2’ saved [940650/940650]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0mobilenet_v1_1.0_224/labels.txt
100 18.2M  100 18.2M    0     0  37.5M      0 --:--:-- --:--:-- --:--:-- 37.4M


In [5]:
import time
import numpy as np
from PIL import Image


num_threads = 1
image_file = 'grace_hopper.bmp'
label_file = 'mobilenet_v1_1.0_224/labels.txt'
input_mean, input_std = 127.5, 127.5


interpreter = tf.lite.Interpreter(
      model_path=tflite_file, num_threads=num_threads)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

# check the type of the input tensor
floating_model = input_details[0]['dtype'] == np.float32

# NxHxWxC, H:1, W:2
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
img = Image.open(image_file).resize((width, height))
img_arr = np.array(img)
imgs = np.array([img_arr for i in range(batch_size)])
print('imgs.shape', imgs.shape)

# add N dim
input_data = imgs

if floating_model:
  input_data = (np.float32(input_data) - input_mean) / input_std

interpreter.set_tensor(input_details[0]['index'], input_data)

start_time = time.time()
interpreter.invoke()
stop_time = time.time()

output_data = interpreter.get_tensor(output_details[0]['index'])
print("output_data.shape", output_data.shape)

with open(label_file, 'r') as f:
  labels = [line.strip() for line in f.readlines()]

results = np.squeeze(output_data[0])

top_k = results.argsort()[-3:][::-1]

for i in top_k:
  if floating_model:
    print('{:08.6f}: {}'.format(float(results[i]), labels[i]))
  else:
    print('{:08.6f}: {}'.format(float(results[i] / 255.0), labels[i]))

print('time: {:.3f}ms'.format((stop_time - start_time) * 1000))

[{'name': 'input', 'index': 0, 'shape': array([  3, 224, 224,   3], dtype=int32), 'shape_signature': array([  3, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'MobilenetEdgeTPU/Predictions/Softmax', 'index': 196, 'shape': array([   3, 1001], dtype=int32), 'shape_signature': array([   3, 1001], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
imgs.shape (3, 224, 224, 3)
output_data.shape (3, 1001)
0.814296: 653:military uniform
0.059763: 458:bow tie, bow-tie, bowtie
0.014469: 835:suit, suit of clothes
time: 150.815ms
